In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv


In [176]:
import tensorflow as tf
#import keras

X_train = pd.read_csv('../input/digit-recognizer/train.csv')
X_test = pd.read_csv('../input/digit-recognizer/test.csv')
X_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [177]:
#Determining the size of input X
n_x, n_xcol = X_train.shape[0], X_train.shape[1]
print(n_x, n_xcol)

42000 785


In [178]:
#Splitting raw data into labels and useful data

y_train = np.array(X_train['label'])
X_train = X_train.drop(['label'], axis = 1)
X_train.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [179]:
#Preprocess the data 
#Normalisation

X_train = X_train / 255.

In [180]:
#Defining Activation and Loss functions

def relu(Z):
    Z = np.where(Z <= 0, 0, Z)
    Z = np.where(Z > 0, 0, Z)
    return Z

def cross_entropy(yhat, y):
    l = -(np.sum(y * np.log(yhat)))
    return l

def softmax(A):
    f = (np.exp(A)/np.sum(np.exp(A)))
    return f

In [181]:
'''#Initialise parameters A, W, b for L layers

def initialise_parameters(X, layer_nos, hidden_layer_nos):
    A_prev = np.array(X).T
    parameters = {}
    parameters['A0'] = A_prev
    layer_caches = []
    n_h = hidden_layer_nos
    for i in range(layer_nos - 1):
        #A_prev = np.array(X_train).T
        n_x = A_prev.shape[0]
        parameters['W' + str(i+1)] = np.random.randn(n_h, n_x) * 0.01
        parameters['b' + str(i+1)] = np.zeros(n_h, 1)
        parameters['Z' + str(i+1)] = np.dot(W, A_prev) + b
        A_prev = relu(Z)
        parameters['A' + str(i+1)] = A_prev
        cache = (parameters['W' + str(i+1)], parameters['b' + str(i+1)], A_prev, parameters['Z' + str(i+1)])  #cache = (W, b, A_prev, Z)
        layer_caches.append(cache)
    
    return caches, parameters'''

"#Initialise parameters A, W, b for L layers\n\ndef initialise_parameters(X, layer_nos, hidden_layer_nos):\n    A_prev = np.array(X).T\n    parameters = {}\n    parameters['A0'] = A_prev\n    layer_caches = []\n    n_h = hidden_layer_nos\n    for i in range(layer_nos - 1):\n        #A_prev = np.array(X_train).T\n        n_x = A_prev.shape[0]\n        parameters['W' + str(i+1)] = np.random.randn(n_h, n_x) * 0.01\n        parameters['b' + str(i+1)] = np.zeros(n_h, 1)\n        parameters['Z' + str(i+1)] = np.dot(W, A_prev) + b\n        A_prev = relu(Z)\n        parameters['A' + str(i+1)] = A_prev\n        cache = (parameters['W' + str(i+1)], parameters['b' + str(i+1)], A_prev, parameters['Z' + str(i+1)])  #cache = (W, b, A_prev, Z)\n        layer_caches.append(cache)\n    \n    return caches, parameters"

In [182]:
#Initialise parameters A, W, b for L layers

def initialise_parameters(X, layer_nos, hidden_layer_nos):
    A_prev = np.array(X.T)
    parameters = {}
    #parameters['A0'] = A_prev
    #layer_caches = []
    n_h = hidden_layer_nos
    for i in range(layer_nos-1):
        #A_prev = np.array(X_train).T
        n_x = A_prev.shape[0]
        parameters['W' + str(i+1)] = np.random.randn(n_h, n_x) * 0.01
        parameters['b' + str(i+1)] = np.zeros((n_h, 1))
        '''parameters['Z' + str(i+1)] = np.dot(W, A_prev) + b
        A_prev = relu(Z)
        parameters['A' + str(i+1)] = A_prev'''
        #cache = (parameters['W' + str(i+1)], parameters['b' + str(i+1)], A_prev, parameters['Z' + str(i+1)])  #cache = (W, b, A_prev, Z)
        #layer_caches.append(cache)
    parameters['W' + str(layer_nos)] = np.random.randn(1, n_h) * 0.01
    parameters['b' + str(layer_nos)] = np.random.randn(1, 1)
    
    return parameters

In [183]:
#Set activation as relu for L-1 layers
#Activation for L-th layer: softmax 
#Loss function: cross entropy
'''
def activation_last_layer(num_layers, parameters):
    L = num_layers
    #linear_cache, parameters = initialise_parameters(X_train, L)
    ZL = np.dot(parameters['W' + str(L)], parameters['A' + str(L-1)]) + parameters['b' + str(L)]
    AL = softmax(ZL)
    
    activation_cache = (ZL, AL)
    #caches = (linear_cache, activation_cache)
    
    return activation_cache'''

"\ndef activation_last_layer(num_layers, parameters):\n    L = num_layers\n    #linear_cache, parameters = initialise_parameters(X_train, L)\n    ZL = np.dot(parameters['W' + str(L)], parameters['A' + str(L-1)]) + parameters['b' + str(L)]\n    AL = softmax(ZL)\n    \n    activation_cache = (ZL, AL)\n    #caches = (linear_cache, activation_cache)\n    \n    return activation_cache"

In [184]:
def forward_propagation(X, num_layers, parameters):
    L = num_layers
    #A_prev = np.array(X.T)
    forward_parameters = {}
    forward_parameters['A' + str(0)] = np.array(X.T)
    for i in range(L - 1):
        forward_parameters['Z' + str(i+1)] = np.dot(parameters['W' + str(i+1)], forward_parameters['A' + str(i)]) + parameters['b' + str(i+1)]
        forward_parameters['A' + str(i+1)] = relu(forward_parameters['Z' + str(i+1)])
        #forward_parameters['A' + str(i+1)] = A_prev
    #linear_cache, parameters = initialise_parameters(X_train, L)
    ZL = np.dot(parameters['W' + str(L)], forward_parameters['A' + str(L-1)]) + parameters['b' + str(L)]
    AL = softmax(ZL)
    forward_parameters['Z' + str(L)] = ZL
    forward_parameters['A' + str(L)] = AL
    #activation_cache = (ZL, AL)
    #caches = (linear_cache, activation_cache)
    
    return forward_parameters

In [185]:
#Encode y into a probability distribution
from sklearn.preprocessing import OneHotEncoder

def oh_encode(y):
    encoded_y = []
    for i in y:
        s = np.zeros(10)
        s[i] = 1
        encoded_y.append(s)
    return np.array(encoded_y)

In [186]:
#Encode y_train

labelled_y = oh_encode(y_train)
print(labelled_y)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [187]:
def compute_cost(labelled_y, forward_parameters):
    cost = 0
    y_hat = forward_parameters['A' + str(L)]
    
    cost += cross_entropy(y_hat, labelled_y)
    
    return cost, labelled_y

In [188]:
def backpropagation(labelled_y, forward_parameters, parameters, layer_nos):
    back_parameters = {}
    L = layer_nos
    yhat = forward_parameters['A' + str(L)]
    dZL= -(labelled_y - yhat)
    dZ = dZL
    dWL = np.dot(dZL, forward_parameters['A' + str(L-1)])
    dbL = dZL
    back_parameters['dZ' + str(L)]
    back_parameters['dW' + str(L)]
    back_parameters['db' + str(L)]
    
    for i in range(L-1, 0, -1):
        dA = (parameters['W' + str(i+1)].T) * dZ
        g_dr = []
        for j in forward_parameters['Z' + str(i)]:
            if (j > 0):
                g_dr.append(1)
            else:
                g_dr.append(0)
        dZ = np.dot(dA, np.array(g_dr))
        dW = np.dot(dZ, forward_parameters['A' + str(i - 1)])
        db = dZ
        back_parameters['dZ' + str(i)]
        back_parameters['dW' + str(i)]
        back_parameters['db' + str(i)]
    
    return back_parameters

In [189]:
def update_parameters(parameters, back_parameters, layer_nos):
    for i in range(L):
        parameters['W' + str(i+1)] -= back_parameters['dW' + str(i+1)]
        parameters['b' + str(i+1)] -= back_parameters['db' + str(i+1)]
    return parameters

In [190]:
def complete_neural_network(X, distributed_y, layer_nos, hidden_layer_nos, Wb_parameters):
    #WbAZ_cache, WbAZ_parameters = initialise_parameters(X, layer_nos, hidden_layer_nos)
    forward_parameters = forward_propagation(X, layer_nos, Wb_parameters)
    cost_loss = compute_cost(distributed_y, forward_parameters)
    derivatives = backpropagation(distributed_y, forward_parameters, Wb_parameters, layer_nos)

    return cost_loss, derivatives

In [191]:
def run_neural_network(X, distributed_y, layer_nos, hidden_layer_nos):
    Wb_parameters = initialise_parameters(X, layer_nos, hidden_layer_nos)
    for i in range(10):
        loss, derivatives = complete_neural_network(X, distributed_y, layer_nos, hidden_layer_nos, Wb_parameters)
        print('Loss after epoch ' + str(i + 1) + ': ' + str(loss))
        Wb_parameters = update_parameters(Wb_parameters, derivatives, layer_nos)

In [192]:
run_neural_network(X_train, labelled_y, 3, 5)

ValueError: shapes (5,784) and (5,42000) not aligned: 784 (dim 1) != 5 (dim 0)